In [1]:
print("ok")

ok


In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone 
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [4]:
PINECONE_API_KEY="ebe14d33-1236-44d4-ad62-6e07bbbf1c78"
PINECONE_API_ENV="gcp-starter"


In [5]:
#Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents


In [6]:
extracted_data = load_pdf("data/")

In [7]:
def text_slipt(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks=text_slipt(extracted_data)
print("Length of chunks",len(text_chunks))

Length of chunks 7020


In [9]:
#download embedding model
def download_hugging_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_embeddings()

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [12]:
print(PINECONE_API_KEY,PINECONE_API_ENV)

ebe14d33-1236-44d4-ad62-6e07bbbf1c78 gcp-starter


In [13]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medicalchatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)
